**<font size=48>Quantitative Economics</font>**<br>
<br>

**Problem Set 6**

**Parham Mohammadi - 401206677**

**Sana Rashidi - 401211522**


# **1**

In [6]:
def samesign(a, b):
    return a * b > 0

def bisect(f, low, high, epsilon=1e-6, max_iter=100000):
    # Ensure that the function values at low and high have opposite signs

    # Perform the bisection method
    for i in range(max_iter):
        # Compute the midpoint
        midpoint = (low + high) / 2

        # Update the interval based on the sign of the function at the midpoint
        if samesign(f(low), f(midpoint)):
            low = midpoint
        else:
            high = midpoint

        # Check if the tolerance level is specified and if the interval size is within tolerance
        if abs(high - low) < epsilon:
            break

    return midpoint



In [7]:
# Example usage:
def f(x):
    return -(x - 3) ** 2

# Interval [a, b]
interval = (0, 6)

# Find the maximum of the function within the interval
max_point = bisect(f, interval[0], interval[1])
max_value = f(max_point)

print("Optimal point within the interval:", max_point)

Optimal point within the interval: 2.9999992847442627


In [33]:
import numpy as np
from autograd import grad, hessian

def Gradient(f, x0, interval, alpha=0.001, epsilon=1e-20, max_iter=5):
    def g(x):
        return grad(f)(x)

    for i in range(max_iter):
        gradient = g(x0)
        print(np.linalg.norm(gradient))
        if np.linalg.norm(gradient) < epsilon:
            break
        else:
            x1 = x0 + alpha * gradient
            for i in range(len(x1)):
                x1[i] = np.clip(x1[i], np.min(interval[i]), np.max(interval[i]))
            x0 = x1
    return x0


In [29]:
import numpy as np
from autograd import grad, hessian

def Gradient_hess(f, x0, interval, alpha=0.0001, epsilon=1e-20, max_iter=10000):
    def g(x):
        return grad(f)(x)

    def hess_f(x):
        return hessian(f)(x)

    for i in range(max_iter):
        gradient = g(x0)
        hess_matrix = hess_f(x0)
        hess_inv = np.linalg.inv(hess_matrix)
        if np.linalg.norm(gradient) < epsilon:
            break
        else:
            x1 = x0 + alpha * (-1)* hess_inv @ gradient
            for i in range(len(x1)):
                x1[i] = np.clip(x1[i], np.min(interval[i]), np.max(interval[i]))
            x0 = x1
    return x0

In [10]:
import numpy as np
import sys
from autograd import grad

def Gradient_vcm(f, x0, interval, alpha=0.001, epsilon=1e-6, max_iter=10000, regularization=1e-6):
    def g(x):
        return grad(f)(x)

    def compute_vcm(x):
        grad_at_x = g(x)
        vcm = np.outer(grad_at_x, grad_at_x)
        return vcm

    for i in range(max_iter):
        gradient = g(x0)
        vcm_matrix = compute_vcm(x0)
        # Add regularization to the diagonal elements of the VCM matrix
        vcm_matrix_reg = vcm_matrix + regularization * np.eye(len(x0))
        try:
            # Check if the VCM matrix is near singular
            if np.linalg.cond(vcm_matrix_reg) < 1/sys.float_info.epsilon:
                # Compute the inverse of the regularized VCM matrix
                vcm_inv = np.linalg.inv(vcm_matrix_reg)
            else:
                # Skip inversion if the matrix is near singular
                continue
        except np.linalg.LinAlgError:
            # Skip inversion if the matrix is singular
            continue

        if np.linalg.norm(gradient) < epsilon:
            break
        else:
            x1 = x0 + alpha  * vcm_inv @ gradient

            for i in range(len(x1)):
                x1[i] = np.clip(x1[i], np.min(interval[i]), np.max(interval[i]))
            x0 = x1
    return x0



In [11]:
# Example usage:
def f(x):
    return -((x[0]-2.8)**2 + (x[1]-4.2)**2)    # Function to minimize: x^2 + y^2

# Initial point
x0 = np.array([2.0, 3.0])
# Interval [a, b]
interval = ([0, 6], [0, 6])

In [12]:
optimal_point_1 = Gradient_hess(f, x0, interval)
print("Optimal point within the interval 1:", optimal_point_1)

optimal_point_2 = Gradient(f, x0, interval)
print("Optimal point within the interval 2:", optimal_point_2)

optimal_point_3 = Gradient_vcm(f, x0, interval)
print("Optimal point within the interval 3:", optimal_point_3)


Optimal point within the interval 1: [2.50571116 3.75856674]
Optimal point within the interval 2: [2.79999972 4.19999958]
Optimal point within the interval 3: [2.80000026 4.20000038]


# **2**

## Library

In [1]:
import numpy as np
from scipy.stats import expon, norm, uniform
from scipy.optimize import minimize

## Constants

In [2]:
sample_size = 500

options = ['Bus', 'Taxi', 'Car']

x1_distribution_params = [(1,1), (4, 1), (2, 1)]
x2_distribution_params = [(1,1), (0.5, 1), (0, 1)]
x3_distribution_params = [(1,1), (2, 1), (2.5, 1)]
x4_distribution_params = [(2,1), (5, 1), (4, 1)]

beta1 = 1
beta2 = 1.5
beta3 = 3
beta4 = -3

epsilon_distribution_params = (0, 0.5)

replication = 1000

## Data Generating Process

In [3]:
def data_generator():
  convenience = np.empty((len(options), sample_size))
  distance = np.empty((len(options), sample_size))
  duration = np.empty((len(options), sample_size))
  cost = np.empty((len(options), sample_size))

  for i in range(len(options)):
    convenience[i, :] = uniform.rvs(
        loc=x1_distribution_params[i][0],
        scale=x1_distribution_params[i][1],
        size=sample_size
    )
    distance[i, :] = expon.rvs(
        loc=x2_distribution_params[i][0],
        scale=x2_distribution_params[i][1],
        size=sample_size
    )
    duration[i, :] = expon.rvs(
        loc=x3_distribution_params[i][0],
        scale=x3_distribution_params[i][1],
        size=sample_size
    )
    cost[i, :] = norm.rvs(
        loc=x4_distribution_params[i][0],
        scale=x4_distribution_params[i][1],
        size=sample_size
    )

  epsilon = norm.rvs(
      loc=epsilon_distribution_params[0],
      scale=epsilon_distribution_params[1],
      size=(len(options), sample_size)
  )

  observed_utility = beta1*convenience + beta2*distance + \
                     beta3*duration + beta4*cost

  utility = observed_utility + epsilon

  choices = np.argmax(utility, axis=0)

  return convenience, distance, duration, cost, choices

## Maximum Simulated Likelihood

In [4]:
def simulated_log_likelihood(
    beta1, beta2, beta3, beta4, choices,
    convenience, distance, duration, cost,
    replication=1000
):
  np.random.seed(0)
  epsilon = norm.rvs(
      loc=epsilon_distribution_params[0],
      scale=epsilon_distribution_params[1],
      size=(len(options), sample_size, replication)
  )

  observed_utility = beta1*convenience + beta2*distance + \
                     beta3*duration + beta4*cost

  observed_utility_expanded = np.tile(
      observed_utility[:, :, np.newaxis],
      (1, 1, replication)
  )

  utility = observed_utility_expanded + epsilon

  simul_choices = np.argmax(utility, axis=0)

  simul_choices_expanded = np.tile(
      simul_choices[np.newaxis, :, :],
      (len(options), 1, 1)
  )

  options_expanded = np.tile(
      np.arange(len(options))[:, np.newaxis, np.newaxis],
      (1, sample_size, replication)
  )

  probs = np.where(
      simul_choices_expanded == options_expanded, 1, 0
  ).mean(axis=2)

  choices_probs = probs[choices, np.arange(choices.shape[0])]

  log_likelihood = np.log(np.maximum(choices_probs, 1e-6)).sum()

  return log_likelihood

## Estimate Parameters

In [5]:
convenience, distance, duration, cost, choices = data_generator()

simulated_log_likelihood(
    beta1, beta2, beta3, beta4, choices,
    convenience, distance, duration, cost,
    replication=1000
)

-40.19895871153179

In [ ]:
objective_function = lambda x: -simulated_log_likelihood(
    x[0], x[1], x[2], x[3], choices,
    convenience, distance, duration, cost,
    replication=10000
)

res = minimize(objective_function, [1, 1, 1, 1], method='Nelder-Mead', tol=1e-7)

print(f"Log Likelihood: {-res.fun:.8f}")
print(f"Estimated Parameters: {res.x}")

Log Likelihood: -53.55186438
Estimated Parameters: [ 0.98075181  1.44434097  2.8702005  -2.86790818]
